In [30]:
import torch
from torch import nn
from torchvision import transforms
import timm
from torchinfo import summary
import pandas as pd
import imageio.v3 as  imageio

from torch.utils.data import Dataset, DataLoader

In [31]:
# Upload data
train_image_bytes = pd.read_csv("../data/02_intermediate/train_image_bytes.csv")

In [32]:
# Build dataset
class MyDataset(Dataset):
    def __init__(self, X_jpeg_bytes, y, features, transforms=None):
        self.X_jpeg_bytes = X_jpeg_bytes
        self.y = y
        self.features = features
        self.transforms = transforms

    def __len__(self):
        return len(self.X_jpeg_bytes)

    def __getitem__(self, index):
        X_sample = {
            'image': self.transforms(
                    image=imageio.imread(self.X_jpeg_bytes[index]),
                )['image'],
            'feature': self.features[index],
        }
        y_sample = self.y[index]
            
        return X_sample, y_sample

In [28]:
TARGETS = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
FEATURES = train_image_bytes.columns.tolist()[2:-14]

In [ ]:
# Build dataloader
# Train
train_dataset = MyDataset(
    train_image_bytes['jpeg_bytes'].values,
    train_image_bytes[TARGETS],
    train_image_bytes[FEATURES],
)

train_dataloader = DataLoader(
        train_dataset,
        batch_size=CONFIG.BATCH_SIZE,
        drop_last=True,
        num_workers=psutil.cpu_count(),
    )
train_dataloader_iter = iter(train_dataloader)

In [ ]:
# Get one batch
# Train
train_dataset = MyDataset(
    train_mask['jpeg_bytes'].values,
    y_train_mask,
    train_features_mask,
    TRAIN_TRANSFORMS,
)

train_dataloader = DataLoader(
        train_dataset,
        batch_size=32,
        drop_last=True,
        num_workers=1,
    )

train_dataloader_example = next(iter(train_dataloader))

In [ ]:
# Feed the motherfucker to the model

In [3]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # ImageNet Normalize Input
        self.normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        # Backbone
        self.backbone = timm.create_model(
                'efficientvit_b1.r288_in1k',
                pretrained=True,
                num_classes=0,
            )
        
        # Features
        self.features = nn.Sequential(
            nn.Linear(len(FEATURES),256),
            nn.GELU(),
            nn.Linear(256,256),
        )
    
        # Label
        self.label = nn.Sequential(
            nn.Linear(256,256),
            nn.GELU(),
            nn.Linear(256,6, bias=False),
        )
        
		# Initialize Weights
        self.initialize_weights()
        
    def initialize_weights(self):
        # Features
        nn.init.kaiming_uniform_(self.features[2].weight)
        # Label
        nn.init.zeros_(self.label[2].weight)
        
    def forward(self, inputs, debug=False):
        if debug:
            embedding = self.backbone(self.normalize(inputs['image'].float() / 255))
            features = self.features(inputs['feature'])
            label = self.label(embedding + features)
            return {
                'features': features,
                'embedding': embedding,
                'label': label,
            }
        else:
            return {
                'label': self.label(
                    self.backbone(self.normalize(inputs['image'].float() / 255)) + self.features(inputs['feature'])
                )
            }

In [9]:
model = Model()

In [ ]:
# Zainicjalizwoać wagi?


In [11]:
model({'image': torch.randn(3,288,288),
            'feature': torch.randn(170)})

ValueError: expected 4D input (got 3D input)

In [7]:
(32, 3, 288, 288)

summary(model=model,
        input_data={
            'image': torch.randn(3,288,288),
            'feature': torch.randn(170)
		},
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [ ]:
summary(
	model

)